# Homework 2

# Set up

## Installing packages

In [1]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [2]:
import os
from google.colab import userdata

try:
    api_key = userdata.get('GOOGLE_API_KEY')
except Exception:
    try:
        api_key = userdata.get('VERTEX_API_KEY')
    except Exception:
        api_key = ""


os.environ["GOOGLE_API_KEY"] = api_key

print(f"API Key Finished {api_key[:5]}...")

API Key Finished AIzaS...


# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [3]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 16.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 18.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 42.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 50.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 50.2MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [4]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [5]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [6]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=api_key,
    temperature=0,
)

# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",          # or "deepseek-reasoner"
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}, '__gemini_function_call_thought_signatures__': {'4cbd5a2b-d7ed-4068-9f5d-03f3f8babded': 'CuIHAb4+9vuhz3TUi5dzHVtXtn19YtPAVzlWwO4Q9tawJZke2yiykArsqJDYBejApJy8j9VY26M5fxD2UJaGg8m0xtc8+CvEusZlvG+6FPCimzIUrwgM0qjs38JYHjkbam9s9tDVUdHrKhYPfZchu3ARBb5H1T8OO5rFcoPN4jheGuetP89NrC2zc5suvI+rUwS48aPfp+t8bBQbbK3bbiAfQxef2TS2kuWoNqEO3ObMp9IdY7PJAtl9AOCS3P1QBt7gJBbOMWd6dQWRtJhjwnEMTWQ85ypUhNMqMKk+eSHyTNuHMsIAzqSBFVldRi4whQ3VSfjup1JizujYpSuX74IBRSbQVT8CMMfzWHvk0Omd1PVPl6n6OG+5j4wY28bJn6adZSb3IDdDGAmb7/ucIn/fEHR4u8ZJRBRfMnVUYuup0FypmsOchYR9oKXiSMLukjksXgsBL1Rsl7vrTEmNNYlVjVEjP3dtwWbpftJ1sDnMmBgu4lypdcswi1GiBjEVyv87eZPUxBDwakogok47UBxPg22uIIOF//TMwXIkm7bGZqVUwljQxr+HRvt4yBtf4XIhITXBp8wH+V2kXgkBCqKpojwuzJYuE2r4iKyl9qkiCefUCTI8JSfpYUn9ZpNw6qvllos0VT8oO2obHlbuyKFVQY05Yeo7/GfMn4C4Hhm6Ysh77aFjHo722dj5JU0Kfd3NcGmB+ZcjRtijKj/ZbLJNSZhZwgz1pWGI2VRmzqdLVTsUjwnGZ+rZ3X9cvik7rVorldgdj28yDk4x3IAO51+ot5Y+17

In [9]:
!pip install langchain-classic

In [10]:
#有报错循环的
import re
import asyncio
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor

tools = mcp_tools

system_prompt = """You are an automated Background Investigation Specialist. Your primary objective is to authenticate a candidate's resume (CV) details by cross-referencing their digital footprint on LinkedIn and Facebook.

### Operational Context
- Assume the current year is 2026 for all "Present" or "Current" roles.
- Experience Calculation: Sum the lengths of individual job entries. Do not subtract the graduation year from 2026. Employment gaps are expected and do not count towards total experience.

### Tool Usage Constraints
- search_facebook_users: Permitted arguments are strictly `q`, `limit`, and `fuzzy`. DO NOT pass location parameters to this tool. Filter by city/location only after retrieving results.
- search_linkedin_people: Permitted arguments include `q`, `location`, `industry`, `limit`, `fuzzy`. You may use the location argument directly here.
- Error Handling: If any API request fails, a minimum of one retry is mandatory.

### Mandatory Verification Workflow

**Phase 1: Resume Extraction**
Isolate and document the candidate's Full Name, Education history, Work history, Location, and Core Skills.

**Phase 2: LinkedIn Validation (Mandatory)**
- Execute search_linkedin_people using the candidate's exact full name.
- Prioritize profiles matching the CV's location or industry. You must evaluate profiles matching either criteria.
- You are required to run get_linkedin_profile on at least 3 distinct person_ids to find the optimal match.
- Evaluate candidates using this scoring matrix:
  * Exact name match: +1
  * Location match: +2 (exact city), +1 (country only)
  * University/School match: +3
  * Degree/Field of study match: +3
  * Company name match (past or present): +4
  * Industry/Headline alignment: +1
  * Skill overlap (minimum 2 shared): +1
- Identify the profile with the highest total. If this peak score is 2 or lower, declare "no confident match found".
- Note on Data Anomalies: A missing `end_year` (`null`) combined with `is_current: false` is a platform glitch, not a resignation. Treat as a MINOR issue.
- Broaden Search: If the highest profile score remains below 4, initiate follow-up searches applying industry or location filters.

**Phase 3: Facebook Validation (Secondary)**
- Execute search_facebook_users using only the full name.
- Sift through results based on geographic location (city/country). Execute get_facebook_profile on 2 to 3 likely profiles.
- Validate these profiles against the confirmed LinkedIn data (look for shared educational institutions, employers, or job titles).
- Absence of a matching Facebook profile is not a severe red flag (deduct a maximum of 0.05 later).

**Phase 4: Discrepancy Analysis**
- Catalog all inconsistencies.
- Categorize as MINOR (typographical errors, 1-year date shifts, data anomalies) or MAJOR (unverified education, missing employers, incorrect degrees, fabricated roles).
- Identity Check: If the best LinkedIn match scores 4 or less AND exhibits a MAJOR discrepancy, deduce an "uncertain match — likely a different person with the same name." Do not penalize for resume fabrication in this scenario.

**Phase 5: Final Evaluation (0.0 to 1.0 Scale)**
- Establish Baseline:
  * 0.75: Education and at least one primary role verified.
  * 0.60: Education verified, employment partially verified.
  * 0.50: Uncertain identity match (potential namesake).
  * 0.30: No viable LinkedIn profile located.
- Apply Modifiers:
  * +0.05 to +0.10: Facebook corroborates key details.
  * 0 to -0.05: No Facebook presence found.
  * -0.02 to -0.05: Per MINOR discrepancy.
  * -0.10 to -0.20: Per MAJOR discrepancy.
  * +0.05: Perfect alignment across CV, LinkedIn, and Facebook.

### Final Output Requirement
Detail your analytical process. The absolute final line of your output must strictly adhere to this format:
FINAL_SCORE: <score>
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

async def process_cvs(cv_list, max_retries=3):
    final_results = []

    print("CV_File\tScore")

    for cv_data in cv_list:
        cv_text = cv_data['text']
        file_name = cv_data.get('file', 'Unknown_File')

        query = f"Analyze the following CV text and verify it using available tools. Ensure your final line is EXACTLY 'FINAL_SCORE: <score>'.\n---\n{cv_text}\n---"

        score = 0.0
        attempts = 0
        final_process_text = ""

        while attempts < max_retries:
            try:
                result = await agent_executor.ainvoke({"input": query})
                output_raw = result['output']

                output_text = ""
                if isinstance(output_raw, list):
                    for item in output_raw:
                        if isinstance(item, dict) and 'text' in item:
                            output_text += item['text']
                        elif isinstance(item, str):
                            output_text += item
                else:
                    output_text = str(output_raw)

                final_process_text = output_text
                match = re.search(r"(?i)final[_ ]score\s*[:=]?\s*([\d\.]+)", output_text)

                if match:
                    score = float(match.group(1))
                    score = max(0.0, min(1.0, score))
                else:
                    score = 0.0

                if score > 0.0:
                    break
                else:
                    print(f"[Warning] {file_name} attempt {attempts + 1} scored 0, retrying...")

            except Exception as e:
                print(f"[Error] {file_name} attempt {attempts + 1} failed: {e}, retrying...")
                score = 0.0
                final_process_text = f"Failed with error: {e}"

            attempts += 1

            if attempts < max_retries:
                await asyncio.sleep(10)

        final_results.append({
            'file': file_name,
            'score': score,
            'process': final_process_text
        })

        print(f"\n[{file_name} - Analysis Process]")
        print(final_process_text)
        print("-" * 40)
        print(f"{file_name}\t{score}\n")

    return final_results
results = await process_cvs(all_cvs)

for res in results:
    print(f"File: {res['file']} | Final Score: {res['score']}")

CV_File	Score

[CV_1.pdf - Analysis Process]
### Phase 1: Resume Extraction

Based on the provided CV, the candidate's details are as follows:
*   **Full Name:** John Smith
*   **Location:** Singapore
*   **Work History:**
    *   Engineer, ByteDance (2020 – Present)
*   **Education:**
    *   McGill University, Bachelor of Science (BSc) in Marketing, Graduated 2009
*   **Core Skills:**
    *   Content Creation
    *   SEO
    *   Social Media

### Phase 2: LinkedIn Validation

I initiated a search for "John Smith" in "Singapore" on LinkedIn. The search returned several results, and I prioritized profiles with the headline "Marketing Professional" as it aligns with the CV's stated profession and education. I selected three potential candidates (IDs: 9, 377, 1636) for detailed profile analysis.

**Profile Evaluation:**

*   **Profile ID 9:**
    *   **Name:** John Smith (+1)
    *   **Location:** Singapore (+2)
    *   **Education:** McGill University, BSc in Marketing, 2009 (+3, +3)
  

<frozen os>:714: RuntimeWarning: coroutine 'ClientResponse.json' was never awaited


[Error] CV_2.pdf attempt 2 failed: unhandled errors in a TaskGroup (1 sub-exception), retrying...

[CV_2.pdf - Analysis Process]
### Investigation Report: Minh Pham

#### Phase 1: Resume Extraction

The following details were extracted from the candidate's CV:
- **Full Name:** Minh Pham
- **Location:** Beijing, China / Hong Kong
- **Work History:**
    - Manager, BCG (2022 – Present)
    - Analyst, Tencent (2013 – 2017)
- **Education:**
    - BSc in Design, The University of Hong Kong (Graduated 2011)
- **Core Skills:**
    - UI/UX Design
    - Prototyping
    - Graphic Design

#### Phase 2: LinkedIn Validation

1.  **Initial Search:** A search was conducted for "Minh Pham" on LinkedIn. The initial results yielded multiple individuals with the same name. Profiles located in Beijing and Hong Kong were prioritized for detailed examination.

2.  **Profile Analysis & Scoring:** Three profiles were selected for in-depth review:
    *   **Profile ID 47 (Minh Pham, Design Professional, Beijin

/usr/local/lib/python3.12/dist-packages/langchain_core/callbacks/base.py:935: RuntimeWarning: coroutine 'ClientResponse.json' was never awaited
  return self.__class__(



[CV_3.pdf - Analysis Process]
### Phase 1: Resume Extraction

*   **Full Name:** Wei Zhang
*   **Location:** Munich, Germany
*   **Hometown:** Sydney, Australia
*   **Work History:**
    *   **Company:** PwC
    *   **Role:** Engineer
    *   **Duration:** 2013 – Present (Current year: 2026)
*   **Education:**
    *   **Degree:** BSc in Consulting
    *   **Institution:** University of Tokyo
    *   **Graduation Year:** 2015
*   **Core Skills:** Data Analysis, Problem Solving, Strategy, PowerPoint

### Phase 2: LinkedIn Validation

1.  **Initial Search:** I initiated a search for "Wei Zhang" in "Munich" on LinkedIn. This yielded multiple results, with several candidates in the "Consulting" industry.

2.  **Profile Analysis & Scoring:** I selected the top three most relevant profiles for detailed analysis.

    *   **Profile ID: 97**
        *   Exact name match ("Wei Zhang"): +1
        *   Location match ("Munich, Germany"): +2
        *   University match ("University of Tokyo"): +3

...

[Error] CV_4.pdf attempt 2 failed: 503 Service Unavailable. {'message': '{\n  "error": {\n    "code": 503,\n    "message": "This model is currently experiencing high demand. Spikes in demand are usually temporary. Please try again later.",\n    "status": "UNAVAILABLE"\n  }\n}\n', 'status': 'Service Unavailable'}, retrying...


/usr/lib/python3.12/asyncio/futures.py:407: RuntimeWarning: coroutine 'ClientResponse.json' was never awaited
  destination.add_done_callback(_call_check_cancel)


[Warning] CV_4.pdf attempt 3 scored 0, retrying...

[CV_4.pdf - Analysis Process]
### Phase 1: Resume Extraction

*   **Full Name:** Rahul Sharma
*   **Location:** Singapore / Philippines
*   **Work History:**
    *   Senior Engineer, Microsoft (2021 - Present)
    *   Consultant, StartupXYZ (2020 - 2023)
*   **Education:**
    *   PhD in Legal Studies, Tsinghua University (Graduated 2021)
*   **Core Skills:**
    *   Legal: Compliance, Litigation, Contract Review
    *   Technical: Web3, Machine Learning, Quantum Computing

### Phase 2: LinkedIn Validation

1.  **Initial Search:** An initial search for "Rahul Sharma" in "Singapore" on LinkedIn yielded 20 potential matches. Given the CV's description as a "Legal Professional," I prioritized profiles matching that industry.

2.  **Profile Evaluation:** I selected the three most plausible profiles for detailed review:
    *   **Profile #2919:** "Legal Professional" - Strongest initial match based on headline.
    *   **Profile #914:** "A

/usr/lib/python3.12/asyncio/locks.py:13: RuntimeWarning: coroutine 'ClientResponse.json' was never awaited
  async def __aenter__(self):



[CV_5.pdf - Analysis Process]
### Phase 1: Resume Extraction

*   **Full Name:** Rahul Sharma
*   **Location:** London, Hong Kong, Singapore (Hometown)
*   **Core Skills:** Machine Learning & AI, Advanced AI Systems, Machine Learning (ML), Natural Language Processing (NLP), TensorFlow, PyTorch, Python.
*   **Work History:**
    *   Senior Engineer at EY (Current)
    *   Consultant at StartupXYZ (2019 – 2021)
    *   Senior Analyst at DataForge (2016 – Present)
    *   Lead Scientist at UrbanFlow (2010 – 2017)
*   **Education:**
    *   PhD in Artificial Intelligence, University of Tokyo (Graduated 2012)

*Initial CV Analysis:* There are immediate red flags within the CV itself. The roles at EY and DataForge are both listed as "Present", and the roles at DataForge (2016-Present) and UrbanFlow (2010-2017) have a one-year overlap.

### Phase 2: LinkedIn Validation

1.  **Initial Search:** I initiated a search for "Rahul Sharma" on LinkedIn, filtering by the primary location mentioned, "

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [11]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [12]:
scores = [0.68, 0.75, 0.68, 0.0, 0.25] # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
